<a href="https://colab.research.google.com/github/matherialist/Python-for-Data-Science-and-AI/blob/master/DOTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import datetime
import numpy as np

In [0]:
features = pd.read_csv('https://d3c33hcgiwev3.cloudfront.net/_47ea8666d262362a0b41164cb497658e_features.zip?Expires=1576454400&Signature=JyMpsLYrTfA8tpzvFaCo~wPBNcRyK0YpF4ZJ~edj~Y73CAe6V6gE3Gfq9HqzQ-WEvtw-BhHCLm9f8~g6zoitn3c1XVM7ySuvhjTuOQW7xOoD-LrLQ7ysSFX4MkHGKFBLR-Q~VrzkwPwKfoqVG-f0gspKVzaVqeCQNtArjmDKufQ_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A', compression='zip', index_col='match_id')

In [3]:
features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,r2_level,r2_xp,r2_gold,r2_lh,r2_kills,r2_deaths,r2_items,r3_hero,r3_level,r3_xp,r3_gold,r3_lh,r3_kills,r3_deaths,r3_items,r4_hero,r4_level,r4_xp,r4_gold,r4_lh,r4_kills,r4_deaths,r4_items,r5_hero,r5_level,r5_xp,r5_gold,r5_lh,r5_kills,...,d4_xp,d4_gold,d4_lh,d4_kills,d4_deaths,d4_items,d5_hero,d5_level,d5_xp,d5_gold,d5_lh,d5_kills,d5_deaths,d5_items,first_blood_time,first_blood_team,first_blood_player1,first_blood_player2,radiant_bottle_time,radiant_courier_time,radiant_flying_courier_time,radiant_tpscroll_count,radiant_boots_count,radiant_ward_observer_count,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,67,3,842,991,10,0,0,4,29,5,1909,1143,10,0,0,8,20,3,757,741,6,0,0,7,105,3,732,658,4,0,...,742,500,2,0,0,8,84,3,958,1003,3,1,0,9,7.0,1.0,9.0,NaN,134.0,-80.0,244.0,2,2,2,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,49,4,1596,993,10,0,1,7,67,4,1506,1502,18,1,0,7,37,3,669,631,7,0,0,7,26,2,415,539,1,0,...,440,583,0,0,0,7,12,4,1470,1622,24,0,0,9,54.0,1.0,7.0,NaN,173.0,-80.0,NaN,2,0,2,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,98,3,1314,775,6,0,0,6,20,3,1297,909,0,1,0,6,27,5,2360,2096,26,1,1,6,4,3,1395,1627,27,0,...,1319,667,4,0,0,7,80,3,1350,1512,25,0,0,7,224.0,0.0,3.0,NaN,63.0,-82.0,NaN,2,5,2,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,30,2,539,539,1,0,0,6,75,5,2037,1139,15,0,0,6,37,2,591,499,0,0,0,6,41,3,712,1075,12,0,...,674,537,1,0,0,7,20,2,510,499,0,0,0,7,NaN,NaN,NaN,NaN,208.0,-75.0,NaN,0,3,2,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,27,2,629,552,0,0,1,7,30,3,884,927,0,1,0,8,72,3,925,1439,16,1,0,11,93,4,1482,880,7,0,...,1815,1275,18,0,0,8,8,4,1119,904,6,0,1,7,-21.0,1.0,6.0,NaN,166.0,-81.0,181.0,1,4,2,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0,2449,0,4,1974,3,63


In [0]:
y_train = features['radiant_win']
X_train = features.loc[:, features.columns != 'radiant_win']

**Подход 1: градиентный бустинг "в лоб"**

In [0]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold, cross_val_score

In [0]:
# Удалим признаки, связанные с итогом матча
X_train = features.drop(['duration', 'radiant_win', 'tower_status_radiant', 
                            'tower_status_dire', 'barracks_status_radiant', 
                            'barracks_status_dire'], axis=1)

In [8]:
# Определим признаки, содержащие пропуски
X_train.columns[X_train.isna().any()].tolist()

['first_blood_time',
 'first_blood_team',
 'first_blood_player1',
 'first_blood_player2',
 'radiant_bottle_time',
 'radiant_courier_time',
 'radiant_flying_courier_time',
 'radiant_first_ward_time',
 'dire_bottle_time',
 'dire_courier_time',
 'dire_flying_courier_time',
 'dire_first_ward_time']

In [0]:
# Заполним пропуски
X_train.fillna(0, inplace=True)

In [0]:
y_train = features['radiant_win']

In [12]:
kf = KFold(n_splits=5, shuffle=True)

for n in [10, 20 , 30, 40, 50, 100, 200]:
    clf = GradientBoostingClassifier(n_estimators=n, verbose=True)
    start_time = datetime.datetime.now()
    score = cross_val_score(clf, X_train, y_train, cv=kf, scoring="roc_auc", n_jobs=-1).mean()

    print('n_estimators =', n)
    print('Score:', score)
    print('Time elapsed:', datetime.datetime.now() - start_time)
    print()

n_estimators = 10
Score: 0.6656492213623775
Time elapsed: 0:00:23.817973

n_estimators = 20
Score: 0.682931639972088
Time elapsed: 0:00:41.710323

n_estimators = 30
Score: 0.6893502778638724
Time elapsed: 0:01:00.978222

n_estimators = 40
Score: 0.6942200315791036
Time elapsed: 0:01:20.625909

n_estimators = 50
Score: 0.6976915069415346
Time elapsed: 0:01:39.477522

n_estimators = 100
Score: 0.7063955209415648
Time elapsed: 0:03:19.278046

n_estimators = 200
Score: 0.713717619208462
Time elapsed: 0:06:30.898982



1. Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?

    Содержат пропуски:
    
 'first_blood_time',

 'first_blood_team',

 'first_blood_player1',

 'first_blood_player2',

 'radiant_bottle_time',

 'radiant_courier_time',

 'radiant_flying_courier_time',

 'radiant_first_ward_time',

 'dire_bottle_time',

 'dire_courier_time',

 'dire_flying_courier_time',

 'dire_first_ward_time'

    Если событие "первая кровь" не успело произойти за первые 5 минут, то признаки first_blood_time, first_blood_team, first_blood_player1, first_blood_player2 принимают пропущенное значение.

2. Как называется столбец, содержащий целевую переменную?

    Целевая переменная - radiant_win.

3. Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Какое качество при этом получилось?

    Для градиентного бустинга с 30 деревьями кросс-валидация длилась 0:01:05.856615,значение ROC AUC при этом 0.6899.

4. Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?

    Есть смысл увеличивать число деревьев, т.к. это ведёт к увеличению метрики ROC AUC. Для увеличения скорости обучения можно ограничить глубину деревьев.

**Подход 2: логистическая регрессия**


In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [0]:
# Удалим признаки, связанные с итогом матча
X_train = features.drop(['duration', 'radiant_win', 'tower_status_radiant', 
                            'tower_status_dire', 'barracks_status_radiant', 
                            'barracks_status_dire'], axis=1)

# Заполним пропуски
X_train.fillna(0, inplace=True)

In [0]:
y_train = features['radiant_win']

In [0]:
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), index=X_train.index, columns=X_train.columns)

In [0]:
def get_score(X_train, y_train):
    kf = KFold(n_splits=5, shuffle=True)

    for i in range(-5, 5):
        C = 10.0 ** i
        clf = LogisticRegression(penalty='l2', C=C, verbose=True)
        start_time = datetime.datetime.now()
        score = cross_val_score(clf, X_train, y_train, cv=kf, scoring="roc_auc", n_jobs=-1).mean()

        print('C =', C)
        print('Score:', score)
        print('Time elapsed:', datetime.datetime.now() - start_time)
        print()

In [0]:
get_score(X_train, y_train)

C = 1e-05
Score: 0.6952211529917117
Time elapsed: 0:00:03.106703

C = 0.0001
Score: 0.7112236342665825
Time elapsed: 0:00:04.607666

C = 0.001
Score: 0.7163290273343642
Time elapsed: 0:00:08.511067

C = 0.01
Score: 0.7164823333610004
Time elapsed: 0:00:11.151747

C = 0.1
Score: 0.7164108862068597
Time elapsed: 0:00:12.017869

C = 1.0
Score: 0.7164465816315506
Time elapsed: 0:00:12.403658

C = 10.0
Score: 0.7165128909892394
Time elapsed: 0:00:12.048100

C = 100.0
Score: 0.7162816517817303
Time elapsed: 0:00:12.379044

C = 1000.0
Score: 0.71651154650236
Time elapsed: 0:00:11.616173

C = 10000.0
Score: 0.716178374079167
Time elapsed: 0:00:12.360493



In [0]:
# Удалим категориальные признаки
X_train = X_train.drop(['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 
                            'r5_hero','d1_hero', 'd2_hero', 'd3_hero', 'd4_hero',                        
                            'd5_hero'], axis=1)

In [0]:
get_score(X_train, y_train)

C = 1e-05
Score: 0.6951440179520855
Time elapsed: 0:00:02.661921

C = 0.0001
Score: 0.7112798721044976
Time elapsed: 0:00:04.002763

C = 0.001
Score: 0.7162553383640671
Time elapsed: 0:00:07.920776

C = 0.01
Score: 0.716609441256771
Time elapsed: 0:00:10.469318

C = 0.1
Score: 0.7166875400608326
Time elapsed: 0:00:10.602719

C = 1.0
Score: 0.7162261162940117
Time elapsed: 0:00:10.568527

C = 10.0
Score: 0.7165548365102998
Time elapsed: 0:00:10.768883

C = 100.0
Score: 0.7160807672352324
Time elapsed: 0:00:10.802860

C = 1000.0
Score: 0.716413712993752
Time elapsed: 0:00:10.912928

C = 10000.0
Score: 0.7163418597087761
Time elapsed: 0:00:10.504198



In [0]:
# Подсчитаем число уникальных героев в датасете
heroes = pd.concat(pd.Series(features[c]) for c in ['r1_hero', 'r2_hero', 'r3_hero', 
            'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'])
heroes.nunique()

108

In [0]:
N = heroes.max()
data = features.drop(['duration', 'radiant_win', 'tower_status_radiant', 
                            'tower_status_dire', 'barracks_status_radiant', 
                            'barracks_status_dire'], axis=1)
data.fillna(0, inplace=True)

In [0]:
# Сформируем мешок слов из id героев
def get_pick(data):
    X_pick = np.zeros((data.shape[0], N))

    for i, match_id in enumerate(data.index):
        for p in range(1, 5):
            X_pick[i, data.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
            X_pick[i, data.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

    X_pick = pd.DataFrame(X_pick, index=data.index, columns=[f"hero_{i}" for i in range(N)])
    return X_pick

In [0]:
X_train = pd.concat([X_train, get_pick(data)], axis=1)

In [0]:
get_score(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


C = 1e-05
Score: 0.6975357434170147
Time elapsed: 0:00:05.610420

C = 0.0001
Score: 0.7204342181316823
Time elapsed: 0:00:06.868471

C = 0.001
Score: 0.7380037956061798
Time elapsed: 0:00:11.393125

C = 0.01
Score: 0.7430343033343642
Time elapsed: 0:00:20.322634

C = 0.1
Score: 0.7431200458929818
Time elapsed: 0:00:26.549308

C = 1.0
Score: 0.7433561472627466
Time elapsed: 0:00:30.969081

C = 10.0
Score: 0.7431346767796236
Time elapsed: 0:00:30.291917

C = 100.0
Score: 0.7429974030460276
Time elapsed: 0:00:29.690078

C = 1000.0
Score: 0.7432306060334142
Time elapsed: 0:00:29.459243

C = 10000.0
Score: 0.7432295745599695
Time elapsed: 0:00:31.570286



In [0]:
model = LogisticRegression(C=1.0)
model.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
features_test = pd.read_csv('https://d3c33hcgiwev3.cloudfront.net/_6866d458468373e164b1e859f3103826_features_test.zip?Expires=1576454400&Signature=a81do3FbUta5aKVWPKgd2CyuHHPvUxm1bV0a2nfzjkLsePz~lme7LkSci0XJHKoot23~2Mwgfa1CrBupHS9qgIWIj200MT2I667vpfzTkuB3XAyjg-yJhdrkmSQs4NlhqvizlZ7TKAQ-HvvdAcJvJFxMiiKy73joXgj4ysBlIM0_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A', compression='zip', index_col='match_id')

In [0]:
X_test = features_test.fillna(0)
X_test = pd.DataFrame(scaler.fit_transform(X_test), index=X_test.index, columns=X_test.columns)
X_test = X_test.drop(['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 
                            'r5_hero','d1_hero', 'd2_hero', 'd3_hero', 'd4_hero',                        
                            'd5_hero'], axis=1)

data = features_test.fillna(0)

X_pick = get_pick(data)
X_test = pd.concat([X_test, X_pick], axis=1)
X_test.head()

,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,r2_gold,r2_lh,r2_kills,r2_deaths,r2_items,r3_level,r3_xp,r3_gold,r3_lh,r3_kills,r3_deaths,r3_items,r4_level,r4_xp,r4_gold,r4_lh,r4_kills,r4_deaths,r4_items,r5_level,r5_xp,r5_gold,r5_lh,r5_kills,r5_deaths,r5_items,d1_level,d1_xp,d1_gold,d1_lh,...,hero_72,hero_73,hero_74,hero_75,hero_76,hero_77,hero_78,hero_79,hero_80,hero_81,hero_82,hero_83,hero_84,hero_85,hero_86,hero_87,hero_88,hero_89,hero_90,hero_91,hero_92,hero_93,hero_94,hero_95,hero_96,hero_97,hero_98,hero_99,hero_100,hero_101,hero_102,hero_103,hero_104,hero_105,hero_106,hero_107,hero_108,hero_109,hero_110,hero_111
match_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6,-2.514875,0.474746,-0.237743,-0.137094,-0.370005,-0.532732,1.003884,0.305165,-0.368985,0.003979,-0.304443,-0.154739,-0.530684,2.625495,0.305627,1.497577,1.986390,2.078705,1.089605,2.548233,-0.576696,-0.935309,-0.337928,-0.451603,-0.179721,0.436965,-0.524238,-0.574233,-0.931472,-0.321391,0.341913,-0.315164,-0.766382,-0.520831,-0.574775,0.286089,-0.458720,-0.207894,-0.433700,-0.689819,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,-2.513895,-1.365630,-1.207913,-1.255660,-1.145090,-0.532732,-0.593200,0.305165,0.567766,0.023887,0.635111,1.185010,-0.530684,-0.578521,-0.525490,0.566666,0.343576,-0.076035,0.303071,-0.526977,1.019230,-0.105828,-1.270990,-0.710585,-0.776060,-1.130442,1.021658,-0.574233,-0.112267,1.557942,1.996586,1.583373,1.601574,-0.520831,-0.574775,-0.943164,-0.458720,-0.585577,-0.388057,0.530232,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,-2.512377,-1.365630,-0.862057,-0.742715,-1.145090,-0.532732,-0.593200,1.898546,-1.305736,-1.375081,-1.179583,-0.824614,-0.530684,-0.578521,-0.941049,-0.364246,-0.399730,-0.445796,-0.820550,1.010628,2.615155,-0.935309,1.528198,1.287019,0.939801,1.444585,-0.524238,-0.574233,-0.112267,0.618275,0.772855,1.152090,1.827094,-0.520831,-0.574775,-1.762666,-0.458720,-0.979220,-1.029242,-1.133474,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,-2.508381,-0.445442,-0.938323,-0.537967,-1.145090,0.968865,1.003884,1.101855,-1.305736,-0.920823,-0.442155,-0.489676,-0.530684,-0.578521,-0.109932,1.497577,1.720667,1.160904,1.988501,-0.526977,-0.576696,-0.520569,0.595135,0.747322,2.398504,2.004373,2.567553,-0.574233,-0.521870,1.557942,1.461999,1.318823,1.939854,-0.520831,-0.574775,-0.123662,0.463282,0.027937,-0.966210,-0.689819,...,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,-2.507087,0.474746,0.038942,-1.059534,-0.812911,-0.532732,-0.593200,0.305165,1.504517,0.945070,-0.302222,-0.043093,-0.530684,1.023487,-0.941049,-0.364246,-0.150166,-0.344552,-0.932912,1.010628,-0.576696,-0.935309,-0.337928,0.214869,-0.164203,-0.010865,-0.524238,-0.574233,-0.112267,-1.261058,-1.221843,-1.035452,-0.653622,-0.520831,-0.574775,-0.123662,0.463282,1.833017,1.155134,0.530232,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
predictions = pd.DataFrame(model.predict_proba(X_test)[:, 1], 
                           index=X_test.index, columns=['radiant_win'])

In [0]:
predictions.describe()

,radiant_win
count,17177.000000
mean,0.517562
std,0.214473
min,0.007986
25%,0.353930
50%,0.520070
75%,0.684218
max,0.995324


1. Какое качество получилось у логистической регрессии над всеми исходными признаками? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

    Получено значение ROC AUC 0.716. Это значение сопоставимо с градиентным бустингом по 200 деревьям. Отсутствие разницы можно объяснить тем, что линейным методам для лучшей работы требуется предобработка данных. Логистическая регрессия работает заметно быстрее градиентного бустинга.
    
2. Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Чем вы можете объяснить это изменение?

    Удаление категориальных признаков никак не повлияло на качество предсказания. В таком случае категориальные признаки можно отнести к шуму, до удаления этих признаков модель оказалась устойчива к этому шуму.

3. Сколько различных идентификаторов героев существует в данной игре?

    Число уникальных героев: 108

4. Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?

    Наилучшее значение AUC-ROC метрики достигается при C = 1.0 и равно 0.743. Добавление мешка слов повысило качество, что объясняется влиянием выбранных героев на исход матча.

5. Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?

    Минимальное значение: 0.008, максимальное значение: 0.995